IMPORTANT NOTE
- if mo ask si maam why this is similar to mine and mag suspect siya na i made it, kay just tell her na youre using mediapipe instead of tensorflow
- 

flaw/weaknesses observed:

evident luminance affecting performance, resulting to erratic tracking which causing it to abruptly stop collecting and considering it as one execution
- should always perform/collect data at a well lit environment
- should apply frame patience mechanism wherein it ignores certain amount of frames before it consider as an execution [APPLIED!]

overload of data
- apply frame buffering(less frame already is sufficient enough to resemble the movement)
- eliminate z coordinate(pose estimation is positional dependant)
- experiment on lessening decimal places (this would sacrifice a certain amount of accuracy, should heavily consider how much of it)


future planned features
 - real-time frame by frame visualization every after execution to check
 - easy-delete button wherein it deletes the current execution collected in case its not good or intended 


How data collection works?
1) show your hands on the camera----------------> data collection starts
2) perform the action you want -----------------> coordinates are collected(coordinates_frame are generated and are compiled in coordinates_movemenet )
3) remove your hand from the camera ------------> data collection stops(coordinates_movemenet is compiled in execution)
4) repeat...

coordinates
- individual X and Y coordinates tracked and collected 
- how it looks like: 0.435. 0.454....etc


coordinates_frame
- compilation of coordinates
- how it looks like: [coordinates, coordinates, coordinates, coordinates..., coordinates,]

execution
- compilation of coordinates_frame, which resembles a movement 
- how it looks like:
[
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
...
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
]

datasets
- compilation of executions
- how it looks like:
[
execution,
execution,
execution,
...
execution,
]


In [ ]:
import cv2
import mediapipe as mp


import os
import numpy as np
import tkinter as tk
from tkinter import simpledialog
from threading import Thread
import re

import random
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# ================================Globals================================
detection_mode = "Multiple"
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
label_map = {}
trained = False

# MediaPipe for hand tracking
mp_hands = mp.solutions.hands
hands_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

known_folder = os.path.join(os.getcwd(), "datasets/")
parent_folder = os.path.dirname(known_folder)
file_path = os.path.join(parent_folder, "output.txt")

cap = cv2.VideoCapture(1)

data_pad = []

# ================================Functions================================
# parameters:
# dataset_noise_collection - by default its false, when True, pose estimation collects for noise data
# noise data - are continuous dataset rather than having executions, not separated by END and START(purpose is to take into account random movements)
# dataset - are datasets separated by END and START (this is are your movements you want to recognize)
def pose_estimation(dataset_noise_collection=False):

    # frame_buffer - maximum frames skipped before a frame is accepted or procesed( to lessen data collected and processed later on) 
    frame_buffer= 8
    frame_buffer_ctr = 0

    # frame_patience - maximum frames to wait before data is collected (this is to take into account erratic interaction with pose estimation, tracking is lost sometimes and might collect incorrect data due to abrupt incorrect detection) - if mawla kalit ang pag detect mo hunong og kalit ang pag collect and ma consider as one execution, so at the middle of the action kay mo conclude dayun siya, so in short maputol ang action.
    frame_patience = 8
    frame_patience_ctr = 0
    
    # coordinates_frame - temporary array for collection of each frames
    coordinates_frame = []

    # coordinates_frame - temporary array for collection or appending of coordinates_frame
    coordinates_movemenet = []

    # execution - temporary array for the collection or appending of coordinates movement
    # this is considered as one execution of the desired action, so set of coordinates_movemenet = one execution
    execution = []
    initialize_padding()

    while True:        
        # frame_buffer_ctr - this increment every frame
        frame_buffer_ctr+=1

        # frame_buffer_ctr>=frame_buffer - if frames has not reached the specified frame buffer then it would not collect any data
        if frame_buffer_ctr>=frame_buffer:   
            # it resets to 0 again after reaching the specified frame buffer counter
            frame_buffer_ctr = 0         
            ret, frame = cap.read()
            if not ret:
                break

            # these are just for detecting the hand and CV configuration.
            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands_detector.process(rgb)

            # this is to check if hands is still in the frame or camera.
            if results.multi_hand_landmarks:    
                # frame_patience_ctr resets when hand is seen in the camera
                frame_patience_ctr=0

# ===========================dataset collection========================================
            # this is used for data collection mode
            if dataset_noise_collection==False:
                if results.multi_hand_landmarks and results.multi_handedness:
                    detected_hands = [hand.classification[0].label for hand in results.multi_handedness]

# STEP 1
# ---------------------------------------------------------------------[COORDINATE COLLECTION]---------------------------------------------------------------------
                    # this is for detecting hand landmarks
                    for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                        # this is to check for the hand labele whether right or left is present
                        hand_label = hand_handedness.classification[0].label 
                        # this is to draw the landmarks
                        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        # for going through the landmarks and enumerating all the coordinates
                        for idx, landmark in enumerate(hand_landmarks.landmark):
                            # coordinates are appended to coordinates_frame in an interleaving manner(x,y,x,y,x,y)
                            coordinates_frame.append(f"{landmark.x:.3f}")
                            coordinates_frame.append(f"{landmark.y:.3f}")

                    # if left is detected then the end of the array is filled with 42 "0.0" this is to fill the missing coordinates of right hand(since not detected, for consistency)
                    if 'Left' in detected_hands and 'Right' not in detected_hands and len(coordinates_frame) != 84:
                        coordinates_frame.extend(data_pad)

                    # this is the opposite, array should start with 42 "0,0" then the rest is the right hand coordinates
                    elif 'Right' in detected_hands and 'Left'not in detected_hands  and len(coordinates_frame) != 84:
                        # right coordinates is temporarily stored
                        temp = coordinates_frame
                        # coordinates_frame array is emptied in preparation for modifying the array
                        coordinates_frame = []                    
                        # data pad contains the 42 "0.0" so this means the array starts with this since this is added first
                        coordinates_frame.extend(data_pad)
                        # it is then followed with the right hand datset
                        coordinates_frame.extend(temp)
#---------------------------------------------------------------------[COORDINATE COLLECTION]---------------------------------------------------------------------

# STEPS 2 (STEP 1 AND STEP 2 REPEATS UNTIL HAND IS NOT DETECTED FOR A CERTAIN AMOUNT OF TIME)
#---------------------------------------------------------------------[FRAME COLLECTION]---------------------------------------------------------------------
                    coordinates_movemenet.append(coordinates_frame)
                    coordinates_frame=[]
#---------------------------------------------------------------------[FRAME COLLECTION]---------------------------------------------------------------------


# STEP 3 EXECUTION IS COLLECTED AND COMPILES ALL FRAME COLLECTED WHEN HAND IS NOT DETECTED ANYMORE
#---------------------------------------------------------------------[FRAME COLLECTION]---------------------------------------------------------------------
            #if hand is not detected 
                else:
                    # if hands is not detected this counts up, if it has not reached the specified amount(frame_patience)
                    if frame_patience_ctr<=frame_patience:
                        frame_patience_ctr += 1

                    # if it had reached the amount of frame_patience then it concludes the action as finished and collected as one execution
                    if frame_patience_ctr >= frame_patience:
                        # it resets again in preparation for the next execution
                        if len(coordinates_movemenet) != 0:    
                            # compiled frames in coordinates movement are considered as one executions
                            execution.append(coordinates_movemenet)
                            # coordinates_movemenet is emptied in preparation for the next execution
                            coordinates_movemenet = []
#---------------------------------------------------------------------[FRAME COLLECTION]---------------------------------------------------------------------


# noise dataset collection is the same with the above only that it doesnt have frame_patience and does not collect by execution, it collects continuously even if you remove your hand from the camera it would not stop collecting to be considered as one exections(for clarifications: it stops collecting frames when hand is not in the camera, however it does not conclude or stop to consider it as one execution) please refer to noise.txt to have more idea of it.
# =========================== noise dataset collection========================================
            else:
                if results.multi_hand_landmarks and results.multi_handedness:
                    detected_hands = [hand.classification[0].label for hand in results.multi_handedness]

                    for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                        hand_label = hand_handedness.classification[0].label 
                        print(f"Detected {hand_label} hand -> {len(coordinates_frame)}")
                        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                        for idx, landmark in enumerate(hand_landmarks.landmark):
                            coordinates_frame.append(f"{landmark.x:.3f}")
                            coordinates_frame.append(f"{landmark.y:.3f}")

                    if 'Left' in detected_hands and 'Right' not in detected_hands and len(coordinates_frame) != 84:
                        coordinates_frame.extend(data_pad)

                    elif 'Right' in detected_hands and 'Left'not in detected_hands  and len(coordinates_frame) != 84:
                        temp = coordinates_frame
                        coordinates_frame = []                    
                        coordinates_frame.extend(data_pad)
                        coordinates_frame.extend(temp)
                        
                    coordinates_movemenet.append(coordinates_frame)
                    coordinates_frame=[]
            cv2.imshow("Recording Face and Hand", frame)            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
    if dataset_noise_collection==True:
        execution.append(coordinates_movemenet)
        text_encode(execution,True)
    else:
        text_encode(execution)


    cap.release()
    cv2.destroyAllWindows()  
    return execution


def inferencing():
    frame_buffer= 8
    frame_buffer_ctr = 0

    frame_patience = 8
    frame_patience_ctr = 0
    
    coordinates_frame = []
    coordinates_movemenet = []
    execution = []
    initialize_padding()

    while True:        
        frame_buffer_ctr+=1
        if frame_buffer_ctr>=frame_buffer:   
            frame_buffer_ctr = 0         
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands_detector.process(rgb)          

            detected_hands = [hand.classification[0].label for hand in results.multi_handedness]

            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                hand_label = hand_handedness.classification[0].label 
                print(f"Detected {hand_label} hand -> {len(coordinates_frame)}")
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                for idx, landmark in enumerate(hand_landmarks.landmark):
                    coordinates_frame.append(f"{landmark.x:.3f}")
                    coordinates_frame.append(f"{landmark.y:.3f}")

            if 'Left' in detected_hands and 'Right' not in detected_hands and len(coordinates_frame) != 84:
                coordinates_frame.extend(data_pad)

            elif 'Right' in detected_hands and 'Left'not in detected_hands  and len(coordinates_frame) != 84:
                temp = coordinates_frame
                coordinates_frame = []                    
                coordinates_frame.extend(data_pad)
                coordinates_frame.extend(temp)
                    
            coordinates_movemenet.append(coordinates_frame)
            coordinates_frame=[]

            # check model input length
            # set it as max length of input, whenever it maxes out remove the first element

            cv2.imshow("Recording Face and Hand", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()  
    return execution

def text_encode(coordinates_data, noise_dataset_collection=False):
    if noise_dataset_collection == True:        
        file_path = get_incremented_filename("datasets/noise/noise.txt")
    else:
        file_path = get_incremented_filename("datasets/datasets/dataset.txt")

    with open(file_path, "w") as file:
        for execution in coordinates_data:
            file.write("START\n")
            for movement_coordinates in execution:
                for coordinate in movement_coordinates:
                    file.write(str(coordinate))
                    file.write("|")
                file.write("\n")
            file.write("END\n")


def text_decode(file_path):
    coordinates_data = []
    with open(file_path, "r") as file:
        lines = file.readlines()

    execution = []
    for line in lines:
        line = line.strip()
        if line == "START":
            execution = []
        elif line == "END":
            coordinates_data.append(execution)
        else:
            str_coords = line.split("|")[:-1]
            float_coords = [float(coord) for coord in str_coords]
            execution.append(float_coords)

    return coordinates_data

def initialize_dummy_dataset():
    frame_coordinate = []
    movement_coordinate = []
    executions = []

    for x in range(60):
        frame_coordinate.append(0.69)

    for i in range(random.randint(8, 15)):    
        for x in range(random.randint(8, 15)):
            movement_coordinate.append(frame_coordinate)
        executions.append(movement_coordinate)
        movement_coordinate = []

    return executions

def initialize_padding():
    if len(data_pad) != 42:
        for ctr in range(42):
            data_pad.append(0.0);

def decode_all_txt_files(folder_path):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            data = text_decode(file_path)
            all_data.append(data) 

    return all_data

def data_padding(input_datasets):
    initialize_padding()
    datasets = input_datasets
    max_len = 0
    padding = []
    padding.extend(data_pad)
    padding.extend(data_pad)


    for classifications in datasets:
        for executions in classifications:
            if max_len <= len(executions):
                max_len = len(executions)

    for classification in datasets:
        for executions in classification:
            while(len(executions) < max_len):
                executions.append(padding)
    print(f"max length of dataset -> {max_len}")    
    return datasets

def noise_preprocessing(input_datsets,dataset_max_length):
    execution = []
    datasets = []
    try:
        for dataset in input_datsets:        
            execution.append(dataset)
            if len(execution) == dataset_max_length:
                execution.append(datasets)
    except:
        return execution
    return execution
    



def data_labelling(input_dataset):
    temp_input_dataset = input_dataset
    label = []
    data = []
    for ctr in range(len(temp_input_dataset)):
        for ctr2 in range(len(temp_input_dataset[ctr])):
            label.append(ctr)
            data.append(temp_input_dataset[ctr][ctr2])
    return [data,label]

def get_incremented_filename(base_path):
    folder = os.path.dirname(base_path)
    basename = os.path.splitext(os.path.basename(base_path))[0]
    ext = os.path.splitext(base_path)[1]

    pattern = re.compile(rf"^{re.escape(basename)}_(\d+){re.escape(ext)}$")

    max_index = 0
    for fname in os.listdir(folder or "."):
        match = pattern.match(fname)
        if match:
            index = int(match.group(1))
            if index > max_index:
                max_index = index

    new_filename = f"{basename}_{max_index + 1}{ext}"
    return os.path.join(folder, new_filename)



def data_preprocessing():
    noise = decode_all_txt_files(parent_folder+"/noise")
    datasets = decode_all_txt_files(parent_folder+"/datasets")

    dataset_padded = data_padding(datasets)
    noise_padded = data_padding(noise)


    label = data_labelling(dataset_padded)

    data_shuffled, label_shuffled = shuffle(label[0], label[1], random_state=42)
    data_shuffled = np.array(data_shuffled)
    label_shuffled = np.array(label_shuffled)



# def coordinates_visualization(coordintes_dataset):
#     canvas_size = 500
#     canvas = np.zeros((canvas_size, canvas_size * 2, 3), dtype=np.uint8)

#     # for execution in coordintes_dataset:
#         # for frame_coordinates in execution[0]:
#         #     # for individual_coordinates in frame_coordinates:
#         #     for individual_coordinates in range(len(frame_coordinates)):
#         #         if individual_coordinates + 1 < len(frame_coordinates):
#         #             if individual_coordinates % 2 != 0:
#         #                 cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)
#     print(len(coordintes_dataset[0]))
#     for frame_coordinates in coordintes_dataset[0]:
#         # for individual_coordinates in frame_coordinates:
#         for individual_coordinates in range(len(frame_coordinates)):
#             if individual_coordinates + 1 < len(frame_coordinates):
#                 if individual_coordinates % 2 != 0:
#                     cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)

#     cv2.imshow("Painted Coordinates", canvas)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     return 0



In [ ]:
noise = decode_all_txt_files(parent_folder+"/noise")
test_noise = noise_preprocessing(noise,5)

In [70]:
len(noise[0][0][0])

84

In [61]:
# collecting dataset
pose_estimation(True)

Detected Right hand -> 0
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Left hand -> 0
Detected Left hand -> 0
Detected Left hand -> 0
Detected Left hand -> 0
Detected Left hand -> 0
Detected Right hand -> 0
Detected Left hand -> 42
Detected Right hand -> 0
Detected Left hand -> 42
Detected Right hand -> 0
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detected Left hand -> 0
Detected Right hand -> 42
Detect

[[[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   '0.484',
   '0.881',
   '0.416',
   '0.825',
   '0.376',
   '0.726',
   '0.351',
   '0.639',
   '0.329',
   '0.567',
   '0.450',
   '0.552',
   '0.442',
   '0.408',
   '0.436',
   '0.319',
   '0.432',
   '0.248',
   '0.506',
   '0.542',
   '0.518',
   '0.385',
   '0.518',
   '0.286',
   '0.517',
   '0.206',
   '0.554',
   '0.559',
   '0.577',
   '0.422',
   '0.584',
   '0.335',
   '0.585',
   '0.266',
   '0.596',
   '0.600',
   '0.629',
   '0.501',
   '0.640',
   '0.433',
   '0.643',
   '0.375'],
  ['0.152',
   '0.771',
   '0.214',
   '0.738',
   '0.262',
   '0.670',
   '0.299',
   '0.618',
   '0.327',
   '0.580',
   '0.234',
   '0.522',
   '0.259',
   

In [40]:
inferencing()

TypeError: 'NoneType' object is not iterable

In [ ]:
# noise collecting dataset
pose_estimation(True)

KeyboardInterrupt: 

In [ ]:
# data preprocessing
data_preprocessing()

max length of dataset -> 64


In [ ]:
len(data_shuffled[0])

64

In [ ]:
num_classification = 2

X_train, X_test, y_train, y_test = train_test_split(
    data_shuffled, label_shuffled, test_size=0.2, random_state=42
)

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(data_shuffled.shape[1], data_shuffled.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classification, activation='softmax'))  # num_classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=8, validation_data=(X_test, y_test))
model.save("my_model.h5")  # or use .keras for newer format


Epoch 1/30
13/13 [==============================] - 7s 155ms/step - loss: 0.6893 - accuracy: 0.5773 - val_loss: 0.6631 - val_accuracy: 0.5600
Epoch 2/30
13/13 [==============================] - 0s 24ms/step - loss: 0.6442 - accuracy: 0.6289 - val_loss: 0.9001 - val_accuracy: 0.6400
Epoch 3/30
13/13 [==============================] - 0s 21ms/step - loss: 0.6400 - accuracy: 0.6598 - val_loss: 0.6371 - val_accuracy: 0.6400
Epoch 4/30
13/13 [==============================] - 0s 20ms/step - loss: 0.5510 - accuracy: 0.7835 - val_loss: 0.4984 - val_accuracy: 0.8400
Epoch 5/30
13/13 [==============================] - 1s 42ms/step - loss: 0.3988 - accuracy: 0.7938 - val_loss: 0.3800 - val_accuracy: 0.8800
Epoch 6/30
13/13 [==============================] - 0s 29ms/step - loss: 0.3224 - accuracy: 0.9072 - val_loss: 0.2659 - val_accuracy: 0.9200
Epoch 7/30
13/13 [==============================] - 0s 21ms/step - loss: 0.3833 - accuracy: 0.8660 - val_loss: 0.3597 - val_accuracy: 0.8800
Epoch 8/30
1

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('my_model.h5')  # or 'my_model.keras'


In [ ]:
print("Model Input Shape:", model.input_shape)


Model Input Shape: (None, 64, 84)
